# I. Modélisation statistique

## I.1. Chargement et préparation des données

In [37]:
!pip install s3fs
!pip install pandas
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' a

In [47]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 44.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 KB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 9.5 MB/s eta 0:00:00:00:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 27.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 1.1 MB/s eta 0:00:0000:01


In [1]:
import pandas as pd
import csv
import os
import s3fs

# Chemin d'accès au fichier CSV
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "marieguibert2"
FILE_KEY_S3 = "Sujet_master/Data/BDD_finale/bdd_finale.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

# Initialiser une liste pour stocker les données
donnees = []
y = []

# Ouvrir le fichier CSV
with fs.open(FILE_PATH_S3, mode="r") as csv_file:
    csv_reader = csv.reader(csv_file)

    # Extraire les noms des colonnes
    colonnes = next(csv_reader)
    
    # Trouver l'index de la colonne 'beneficiaire_trans_eco'
    index_beneficiaire = colonnes.index('beneficiaire_trans_eco')
    
    # Lire chaque ligne du fichier CSV
    for row in csv_reader:

        # Exclure la colonne 'beneficiaire_trans_eco' des variables explicatives
        donnees_x = [row[i] for i in range(len(row)) if i != index_beneficiaire]
        # Ajouter la valeur de la colonne 'beneficiaire_trans_eco' à y
        y.append(int(row[index_beneficiaire]))
        donnees.append(donnees_x + [y])  # Ajouter les données X et y à la liste

# Créer un DataFrame à partir des données
donnees_df = pd.DataFrame(donnees, columns=colonnes)
donnees_df = donnees_df.set_index('code_insee')

# Afficher les premières lignes du DataFrame
print(donnees_df.head())

KeyError: 'AWS_S3_ENDPOINT'

In [72]:
# Convertir les listes en tableaux numpy
X = np.array(X)
y = np.array(y)

In [73]:
print("Dimensions de X:", X.shape)
print("Dimensions de y:", y.shape)

Dimensions de X: (1189, 29)
Dimensions de y: (1189,)


In [74]:
np.unique(y)

array([0, 1])

In [78]:
donnees_df.dtypes

beneficiaire_trans_eco                object
moyenne_conso_agri                    object
moyenne_conso_indus                   object
moyenne_conso_tertiaire               object
moyenne_conso_residentiel             object
moyenne_conso_totale                  object
emissions_ges                         object
nb_actes_france_renov                 object
friche                                object
ecoquartiers                          object
nb_residences_secondaires             object
beneficiaire_prog                     object
climat                                object
gridens7                              object
superf_choro                          object
gare_tgv                              object
part_trajets_voiture                  object
med_disp                              object
p_pop                                 object
CSP_maire                             object
com_variation_encours_dette_ha_pct    object
nb_inactifs                           object
nb_actifs 

In [79]:
donnees_df.isna().sum()

beneficiaire_trans_eco                0
moyenne_conso_agri                    0
moyenne_conso_indus                   0
moyenne_conso_tertiaire               0
moyenne_conso_residentiel             0
moyenne_conso_totale                  0
emissions_ges                         0
nb_actes_france_renov                 0
friche                                0
ecoquartiers                          0
nb_residences_secondaires             0
beneficiaire_prog                     0
climat                                0
gridens7                              0
superf_choro                          0
gare_tgv                              0
part_trajets_voiture                  0
med_disp                              0
p_pop                                 0
CSP_maire                             0
com_variation_encours_dette_ha_pct    0
nb_inactifs                           0
nb_actifs                             0
dependance_eco                        0
abstention_municipales                0


## I.2. Modélisation

### I.2.1. Régression logistique

#### A) Construction du modèle et prévisions

Nous allons d'abord diviser notre jeu de données en jeux d'apprentissage et de test.

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) # random_state=0 pour avoir le même mélange de données à chaque fois

In [59]:
X_train.shape, y_train.shape

((891, 29), (891,))

In [84]:
X_test.shape, y_test.shape

((298, 29), (298,))

Nous allons maintenant définir notre modèle de régression logistique

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [82]:
modele_reg_log = Pipeline([
    ("clf", LogisticRegression())
])

On entraîne le modèle sur les données

In [88]:
modele_reg_log.fit(X_train, y_train)

/home/onyxia/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('clf', LogisticRegression())])

On cherche à prédire de nouvelles données grâce aux données test

In [89]:
pred_reg_log = modele_reg_log.predict(X_test)

ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [ ]:
pred_reg_log.shape

#### B) Mesure de la performance

Nous allons maintenant évaluer la performance de notre modèle

In [ ]:
score_reg_log = modele_reg_log.score(X_test, y_test)
print("Le score du modèle est : ", score_reg_log)

Nous allons construire la matrice de confusion pour avoir plus de précisions : 

In [ ]:
from sklearn import metrics

confusion_reg_log = metrics.confusion_matrix(y_test,pred_reg_log)
print("Matrice de confusion")
print(confusion_reg_log)

In [ ]:
# A MODIFIER
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

Text(0.5,257.44,'Predicted label');

A faire : 
- courbe ROC
- matrice de confusion à modifier

### I.2.2. Arbres

### I.2.3. Forêts Aléatoires

Nous allons d'abord séparer notre jeu de données en échantillon test et apprentissage

In [85]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [86]:
X_train.shape, y_train.shape

((891, 29), (891,))

In [87]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

p = Pipeline([
    ("clf", RandomForestClassifier())
])

p.fit(X_train, y_train)
p.score(X_test, y_test)

0.8187919463087249

## I.3. Sélection du meilleur modèle